## If we run the whole notebook we will get four output files. only advection rewiring, only consenus rewiring, consenus-advection with variable ratios of probability, and consensus advection with equal probability but now with the additional random rewiring probability variable (from 0 to 1). 

## So in total 4. You will understand it, because everytime you cross one window that runs the simulation it will give you message that will update you on the progress, i.e. we are at 1 iteration....

In [1]:
# Add project's path to system path
import os
import sys
try: 
    proj_path
except NameError:
    os.chdir('../')
    proj_path = os.getcwd()
    if proj_path not in sys.path:
        sys.path.append(proj_path)

# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy import linalg
from codebase import utils, rewiring, rewiring_iterations # custom-made functions

## Rewire only with consesus or advection

In this preliminary stage we run the rewiring algorithm separately for the consenus algorithim rewiring only the in-degrees of candidate nodes & the advection algorithm rewiring only the out-degrees. We save the rewired digraphs  

In [2]:
#the parameters we are going to use for advection and consensus

taus = [1]
pRandRewires = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

nVertices = 100
edges = int(np.round(2 * np.log(nVertices) * (nVertices - 1), decimals=0))

nRewiringsVec = [500, 1000, 2000, 4000]
iterations = 100

directorySave = 'data/'

In [3]:
#rewire the in-degrees using the consensus algorithm
rewMeth = 'in' #rewire the in-degrees
alg = 'consensus' #use the consensus algorithm
A = {}

print('rewiring algorithm is %s and method is %s'%(alg,rewMeth))
filePathSave = directorySave +'A_'+alg+'_'+rewMeth+'_diff_p.pckl'

A = rewiring_iterations.run_dynamics_iterations(
            nVertices,edges, pRandRewires, taus, nRewiringsVec, rewMeth, alg,iterations, weightDistribution = 'binary')

utils.save_var(A,filePathSave)

rewiring algorithm is consensus and method is in
we are at 1 iteration 
we are at 2 iteration 


KeyboardInterrupt: 

In [4]:

#rewire the out-degrees using the advection algorithm
rewMeth = 'out'
alg = 'advection'
A = {}

print('rewiring algorithm is %s and method is %s'%(alg,rewMeth))
filePathSave = directorySave +'A_'+alg+'_'+rewMeth+'_diff_p.pckl'

A = rewiring_iterations.run_dynamics_iterations(
            nVertices,edges, pRandRewires, taus, nRewiringsVec, rewMeth, alg,iterations, weightDistribution = 'binary')

utils.save_var(A,file_path_save)



rewiring algorithm is advection and method is out
we are at 1 iteration 
we are at 2 iteration 
we are at 3 iteration 
we are at 4 iteration 
we are at 5 iteration 
we are at 6 iteration 
we are at 7 iteration 
we are at 8 iteration 
we are at 9 iteration 
we are at 10 iteration 
we are at 11 iteration 
we are at 12 iteration 
we are at 13 iteration 
we are at 14 iteration 
we are at 15 iteration 
we are at 16 iteration 
we are at 17 iteration 
we are at 18 iteration 
we are at 19 iteration 
we are at 20 iteration 
we are at 21 iteration 
we are at 22 iteration 
we are at 23 iteration 
we are at 24 iteration 
we are at 25 iteration 
we are at 26 iteration 
we are at 27 iteration 
we are at 28 iteration 
we are at 29 iteration 
we are at 30 iteration 
we are at 31 iteration 
we are at 32 iteration 
we are at 33 iteration 
we are at 34 iteration 
we are at 35 iteration 
we are at 36 iteration 
we are at 37 iteration 
we are at 38 iteration 
we are at 39 iteration 
we are at 40 iteration 

## Run the algorithm that does both advection and consensus

In [2]:
# THE ORIGINAL FUNCTIONS ARE IN REWIRING_ITERATIONS.PY FUNCTIONS I.E. RUN_DYNAMICS_STEPS , RUN_DYNAMICS_DIFF_VALUES, 
# AND RUN_DYNAMICS_ITERATIONS. 
#THESE VERSIONS WITH THE 2 AT THE END ARE WHEN WE CHANGE THE RATIOS OF BOTH ADVECTION AND CONSENSUS IN THE REWIRING.
#THE NEXT (ENDING WITH 3) IS WHEN ADVECTION AND CONSENSUS ARE EQUALLY APPLIED WITH 50% PROB, AND WE PARAMETRIZE 
#THE AMOUNT OF RANDOM REWIRING
#WHAT WE WANT IS TO CHANGE THE ORIGINAL FUNCTIONS IN REWIRING_ITERATIONS.PY SO THAT WE INCLUDE THESE CASES
#AND WE DO NOT HAVE TO COPY PASTE THE FUNCTIONS WITH  A SLIGHT CHANGE HERE. 



def run_dynamics_steps2(
    n_nodes, edges, tau,p_adv, n_rewirings_vec, **kwargs
):
 

    AInit = rewiring.generate_rand_Adj(n_nodes, edges, **kwargs)
    A = {}
    A[0] = AInit
    #n_rewirings_vec =  np.insert(n_rewirings_vec, 0, 0, axis=0)

    for ind, n_rewirings in enumerate(n_rewirings_vec):

        if ind == 0:
            AInit = A[0]
            subtract = 0
        else:
            subtract = n_rewirings_vec[ind - 1]

        rewirings = n_rewirings - subtract
        A[n_rewirings] = rewiring.run_dynamics_advection_consensus(AInit, rewirings, tau, p_adv)

        # initialize for the next iteration
        AInit = A[n_rewirings]

    return A

In [3]:
def run_dynamics_diff_values2(
    n_nodes, edges, tau_vec, p_adv_vec, n_rewirings_vec, **kwargs
):

    A = {}
    for p_adv in p_adv_vec:
        for tau in tau_vec:

            A[(p_adv, tau)] = run_dynamics_steps2(n_nodes, edges, tau,p_adv, n_rewirings_vec, **kwargs)

    return A


In [4]:
def run_dynamics_iterations2(
    n_nodes, edges, tau_vec,p_adv_vec, n_rewirings_vec, iterations, **kwargs
):

    A_all = {}
    for it in np.arange(iterations):
        print('we are at %d iteration ' % (it + 1))
        A_all[it + 1] = run_dynamics_diff_values2(n_nodes, edges, tau_vec,p_adv_vec, n_rewirings_vec, **kwargs)

    return A_all


In [5]:
taus = [1]
p_adv_vec = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

n_vertices = 100
edges = int(np.round(2 * np.log(n_vertices) * (n_vertices - 1), decimals=0))

n_rewirings_vec = [500, 1000, 2000, 4000,8000]
iterations = 100

directory_save = 'data/'

In [6]:
A = {}

file_path_save = directory_save +'A_cons_adv_diff_p.pckl'

A = run_dynamics_iterations2(
            n_vertices,edges, taus, p_adv_vec, n_rewirings_vec, iterations, weightDistribution = 'binary')

utils.save_var(A,file_path_save)

we are at 1 iteration 
we are at 2 iteration 
we are at 3 iteration 
we are at 4 iteration 
we are at 5 iteration 
we are at 6 iteration 
we are at 7 iteration 
we are at 8 iteration 
we are at 9 iteration 
we are at 10 iteration 
we are at 11 iteration 
we are at 12 iteration 
we are at 13 iteration 
we are at 14 iteration 
we are at 15 iteration 
we are at 16 iteration 
we are at 17 iteration 
we are at 18 iteration 
we are at 19 iteration 
we are at 20 iteration 
we are at 21 iteration 
we are at 22 iteration 
we are at 23 iteration 
we are at 24 iteration 
we are at 25 iteration 
we are at 26 iteration 
we are at 27 iteration 
we are at 28 iteration 
we are at 29 iteration 
we are at 30 iteration 
we are at 31 iteration 
we are at 32 iteration 
we are at 33 iteration 
we are at 34 iteration 
we are at 35 iteration 
we are at 36 iteration 
we are at 37 iteration 
we are at 38 iteration 
we are at 39 iteration 
we are at 40 iteration 
we are at 41 iteration 
we are at 42 iteration 
w

## Do advection with consensus at 50% with different pRand

In [2]:
def run_dynamics_steps3(
    n_nodes, edges, tau,p_adv,p_rnd_rewire, n_rewirings_vec, **kwargs
):
 

    AInit = rewiring.generate_rand_Adj(n_nodes, edges, **kwargs)
    A = {}
    A[0] = AInit
    #n_rewirings_vec =  np.insert(n_rewirings_vec, 0, 0, axis=0)

    
    for ind, n_rewirings in enumerate(n_rewirings_vec):

        if ind == 0:
            AInit = A[0]
            subtract = 0
        else:
            subtract = n_rewirings_vec[ind - 1]

        rewirings = n_rewirings - subtract
        A[n_rewirings] = rewiring.run_dynamics_advection_consensus_with_random_rewire(AInit, rewirings, tau, p_adv,p_rnd_rewire)

        # initialize for the next iteration
        AInit = A[n_rewirings]

    return A

In [3]:
def run_dynamics_diff_values3(
    n_nodes, edges, tau, p_adv,p_rnd_vec, n_rewirings_vec, **kwargs
):

    A = {}
    for p_rnd in p_rnd_vec:
        A[(p_rnd, tau)] = run_dynamics_steps3(n_nodes, edges, tau,p_adv,p_rnd, n_rewirings_vec, **kwargs)

        
    return A

In [4]:
def run_dynamics_iterations3(
    n_nodes, edges, tau,p_adv,p_rnd_vec, n_rewirings_vec, iterations, **kwargs
):

    A_all = {}
    for it in np.arange(iterations):
        print('we are at %d iteration ' % (it + 1))
        A_all[it + 1] = run_dynamics_diff_values3(n_nodes, edges, tau,p_adv,p_rnd_vec, n_rewirings_vec, **kwargs)

    return A_all

In [5]:
tau = 1
p_adv = 0.5
p_rnd_vec = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

n_vertices = 100
edges = int(np.round(2 * np.log(n_vertices) * (n_vertices - 1), decimals=0))


n_rewirings_vec = [500, 1000, 2000, 4000,8000]
iterations = 100

directory_save = 'data/'

In [6]:
A = {}
wD = 'binary'
file_path_save = directory_save +'A_cons_adv_50_pRand.pckl'

A = run_dynamics_iterations3(
            n_vertices,edges, tau, p_adv,p_rnd_vec, n_rewirings_vec, iterations, weightDistribution = wD)

utils.save_var(A,file_path_save)

we are at 1 iteration 
we are at 2 iteration 
we are at 3 iteration 
we are at 4 iteration 
we are at 5 iteration 
we are at 6 iteration 
we are at 7 iteration 
we are at 8 iteration 
we are at 9 iteration 
we are at 10 iteration 
we are at 11 iteration 
we are at 12 iteration 
we are at 13 iteration 
we are at 14 iteration 
we are at 15 iteration 
we are at 16 iteration 
we are at 17 iteration 
we are at 18 iteration 
we are at 19 iteration 
we are at 20 iteration 
we are at 21 iteration 
we are at 22 iteration 
we are at 23 iteration 
we are at 24 iteration 
we are at 25 iteration 
we are at 26 iteration 
we are at 27 iteration 
we are at 28 iteration 
we are at 29 iteration 
we are at 30 iteration 
we are at 31 iteration 
we are at 32 iteration 
we are at 33 iteration 
we are at 34 iteration 
we are at 35 iteration 
we are at 36 iteration 
we are at 37 iteration 
we are at 38 iteration 
we are at 39 iteration 
we are at 40 iteration 
we are at 41 iteration 
we are at 42 iteration 
w